In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nietzsche-texts/nietzsche.txt
/kaggle/input/nyt-comments/CommentsFeb2018.csv
/kaggle/input/nyt-comments/ArticlesFeb2017.csv
/kaggle/input/nyt-comments/CommentsApril2018.csv
/kaggle/input/nyt-comments/ArticlesJan2017.csv
/kaggle/input/nyt-comments/ArticlesMay2017.csv
/kaggle/input/nyt-comments/CommentsJan2017.csv
/kaggle/input/nyt-comments/CommentsMarch2017.csv
/kaggle/input/nyt-comments/CommentsMay2017.csv
/kaggle/input/nyt-comments/CommentsMarch2018.csv
/kaggle/input/nyt-comments/CommentsApril2017.csv
/kaggle/input/nyt-comments/ArticlesMarch2017.csv
/kaggle/input/nyt-comments/ArticlesApril2017.csv
/kaggle/input/nyt-comments/CommentsFeb2017.csv
/kaggle/input/nyt-comments/ArticlesJan2018.csv
/kaggle/input/nyt-comments/ArticlesFeb2018.csv
/kaggle/input/nyt-comments/ArticlesMarch2018.csv
/kaggle/input/nyt-comments/CommentsJan2018.csv
/kaggle/input/nyt-comments/ArticlesApril2018.csv


# CHARACTER LEVEL TEXT GENERATION

## Loading of Data

In [ ]:
# Loading the data
import keras
import numpy as np

In [ ]:
#path= keras.utils.get_file('nietzsche.txt',origin='https://s3.amazonaws.com/text-datasets.nietzsche.txt')
text=open('/kaggle/input/nietzsche-texts/nietzsche.txt').read().lower()

In [ ]:
print(text)

In [ ]:
print(' Corpus length:',len(text))

## Step 2:- vectorization of data

In [ ]:
maxlen=60
steps=3
sentences=[]
next_chars=[]
for i in range(0,len(text)-maxlen,steps):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])


In [ ]:
print(sentences[0]," ",next_chars[0])


In [ ]:
print(sentences[1]," ",next_chars[1])

In [ ]:
print(sentences[2]," ",next_chars[2])

In [ ]:
print(" number of sequence:",len(sentences))

In [ ]:
print(" number of characters:",len(next_chars))

In [ ]:
chars=sorted(list(set(text)))
print(chars)

In [ ]:
len(chars)

In [ ]:
char_indices=dict((char,chars.index(char)) for char in chars)
print(char_indices)

In [ ]:
print("Vectorization....")
x=np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
x.shape

In [ ]:
y=np.zeros((len(sentences),len(chars)),dtype=np.bool)
y

In [ ]:
for i, sentence in enumerate(sentences):
    for t,char in enumerate(sentence):
        x[i,t,char_indices[char]]=1
    y[i,char_indices[char]]=1  

In [ ]:
x

In [ ]:
y

# Step 3:- Modelling the data

In [ ]:
from keras import layers
model=keras.models.Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars),activation='softmax'))

In [ ]:
model.summary()

In [ ]:
opt=keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=opt)

In [ ]:
y

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random 
import sys
for i in range(1,30):
    print('epoch: ',i)
    model.fit(x,y,batch_size=128,epochs=1)
   
for temp in [0.2,0.5,1.0,1.2]:
        start_index=random.randint(0,len(text)-60-1)
        generated_text=text[start_index:start_index+maxlen]
        print( "temp- " ,temp)
        print("sentence:-",generated_text)
        generate=""
        for i in range(400):
            sampled=np.zeros((1,maxlen,len(chars)))
            for t,char in enumerate(generated_text):
                sampled[0,t,char_indices[char]]=1

            preds=model.predict(sampled,verbose=0)[0]
            #print(preds.shape)
            next_index = sample(preds, temp)
            next_char = chars[next_index]
            #print(next_char)
            generated_text=generated_text[1:]+ next_char
            generate=generate+next_char
        print("...Generated: ", generate)
        print()

# TEXT GENERATION USING LSTM

In [2]:
#Loading necessary libraries
#libraries for data handling
import pandas as pd
import numpy as np
import string, os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [8]:
import tensorflow
tensorflow.random.set_seed(2)
from numpy.random import seed

seed(1)

In [6]:
from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

In [11]:
curr_dir = '/kaggle/input/nyt-comments/'
all_headlines = []
for filename in os.listdir(curr_dir):
    print(filename)
    if 'Articles' in filename:
        
        article_df = pd.read_csv(curr_dir + filename)
        all_headlines.extend(list(article_df.headline.values))
        break
all_headlines = [line for line in all_headlines if line!= "Unknown"]
print(all_headlines[:10])

CommentsFeb2018.csv
ArticlesFeb2017.csv
['N.F.L. vs. Politics Has Been Battle All Season Long', 'Voice. Vice. Veracity.', 'A Stand-Up’s Downward Slide', 'New York Today: A Groundhog Has Her Day', 'A Swimmer’s Communion With the Ocean', 'Trail Activity', 'Super Bowl', 'Trump’s Mexican Shakedown', 'Pence’s Presidential Pet', 'Fruit of a Poison Tree']


In [66]:
len(all_headlines)

829

In [13]:
def clean_text(txt):
    txt = "".join(t for t in txt if t not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt
corpus = [clean_text(x) for x in all_headlines]
print(corpus[:10])

['nfl vs politics has been battle all season long', 'voice vice veracity', 'a standups downward slide', 'new york today a groundhog has her day', 'a swimmers communion with the ocean', 'trail activity', 'super bowl', 'trumps mexican shakedown', 'pences presidential pet', 'fruit of a poison tree']


In [50]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    print("total_words:-", total_words)
    ## convert data to a token sequence 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])
        for i in range(1, len(token_list[0])):
            n_gram_sequence = token_list[0][:i+1]
        input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
print(inp_sequences[:10])

total_words:- 2288
[[660, 117, 72, 73, 661, 662, 63, 29, 210], [211, 663, 664], [2, 665, 666, 345], [11, 27, 28, 2, 667, 73, 153, 90], [2, 668, 669, 12, 1, 670], [346, 671], [212, 213], [19, 672, 673], [347, 348, 674], [675, 4, 2, 349, 676]]


In [52]:
max_sequence_len = max([len(x) for x in inp_sequences])
print(max_sequence_len) 

17


In [61]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    #t(predictors," ",label)
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

In [62]:
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [65]:
print(predictors.shape)

(829, 16)


In [64]:
print(label.shape)

(829, 2288)


In [67]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    # ----------Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    # ----------Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    # ----------Add Output Layer
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [60]:
model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 16, 10)            22880     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 2288)              231088    
                                                                 
Total params: 298,368
Trainable params: 298,368
Non-trainable params: 0
_________________________________________________________________


In [68]:
model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [92]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        print(max(predicted[0]))
        print(next(i for i, x in enumerate(predicted[0]) if x == max(predicted[0])))
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == next(i for i, x in enumerate(predicted[0]) if x == max(predicted[0])):
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [93]:
print (generate_text("india and pakistan", 3, model, max_sequence_len))
print (generate_text("president trump", 3, model, max_sequence_len))
print (generate_text("united states", 4, model, max_sequence_len))
print (generate_text("donald trump", 2, model, max_sequence_len))
print (generate_text("new york", 3, model, max_sequence_len))
print (generate_text("science and technology", 5, model, max_sequence_len))


0.04584018
834
0.05971363
1848
0.10290712
62
India And Pakistan Ring Bridges Love
0.12520811
859
0.08023845
1849
0.11247846
90
President Trump Dizziness Intrigue Day
0.046770703
504
0.1766497
90
0.036006186
732
0.11506696
90
United States Stories Day Mitch Day
0.087685525
504
0.10985881
338
Donald Trump Stories Kind
0.07144982
78
0.16310106
267
0.07579539
136
New York Vote Know Night
0.029884014
2084
0.056673773
2068
0.051909488
567
0.071469516
133
0.08774651
144
Science And Technology Paris Preening Crisis 2017 Month
